# **11. IB API를 사용하여 주문, 포지션 및 포트폴리오 관리하기**  

알고리즘 트레이딩에서 주문, 포지션 및 포트폴리오 데이터를 효율적으로 관리하는 것은 매우 중요합니다. 다행히도 우리는 Python을 사용하여 이를 모두 수행할 수 있습니다. 주문 관리는 새로운 거래 실행, 기존 주문 취소, 시장 상황 변화 또는 트레이딩 전략 변경에 따른 주문 업데이트 등의 다양한 활동을 포함합니다.  

포지션 관리는 실시간 포지션 데이터를 모니터링하고 분석하여 **손익(PnL)** 을 추적하는 작업을 포함합니다. 개별 거래의 성과에 대한 즉각적인 통찰을 제공하여 트레이더가 보유, 매도 또는 조정 여부를 결정하는 데 도움을 줍니다. 또한 실시간(또는 거의 실시간) 포트폴리오 데이터는 실시간(또는 거의 실시간) 위험 통계를 생성하여 전반적인 위험 관리를 향상시킬 수 있습니다.  

포트폴리오 데이터 관리는 포트폴리오의 성과를 평가하고, 위험 노출도를 이해하며, 최적의 수익을 위해 전략적 조정을 수행하는 종합적인 분석을 포함합니다. 특히 마진으로 주식을 거래하거나 선물을 거래할 때 손실 포지션을 유지하는 데 따른 금융 비용과 기회 비용이 존재하기 때문에 더욱 중요합니다.  

IB API는 일관된 **요청-콜백 패턴**을 사용하며, 이를 통해 트레이딩 앱의 여러 측면에서 활용할 수 있습니다. 예를 들어, 주문 관리, 포지션 관리, 포트폴리오 세부 정보 액세스 등에서 사용됩니다. 이 패턴을 사용하면 주문을 배치하거나 수정하는 요청을 시작하고, 현재 포지션 데이터를 검색하거나 포트폴리오 정보를 수집할 수 있으며, 그에 대한 응답을 처리하는 콜백 함수가 작동합니다. 이 장에서는 이 패턴을 계속 사용할 것입니다.  

이 장이 끝날 때쯤이면 Python을 통해 IB API에 주문을 제출하고 수정할 수 있을 뿐만 아니라, 유동성, 마진 요구 사항 및 오픈 포지션을 포함한 주요 포트폴리오 정보를 가져오고, 포트폴리오의 손익을 계산할 수 있게 될 것입니다.  

이 장에서 다룰 내용은 다음과 같습니다:  
- IB API를 사용하여 주문 실행하기  
- 주문이 실행된 후 관리하기  
- 포트폴리오 세부 정보 가져오기  
- 포지션 및 포지션 세부 정보 검사하기  
- 포트폴리오 손익 계산하기  

---

## **IB API를 사용하여 주문 실행하기**  

이전 **10장 (Interactive Brokers Python API 설정하기)** 에서 우리는 `contract` 및 `order` 객체를 생성했습니다. 이를 사용하여 IB API를 통해 거래를 실행할 수 있습니다. 그러나 거래를 실행하기 전에 **다음 주문 ID (nextValidOrderId)** 개념을 이해해야 합니다.  

`nextValidOrderId`는 각 주문을 위한 고유 식별자입니다. 트레이딩 앱이 최대 32개의 인스턴스를 병렬로 실행할 수 있으므로, 이 식별자는 개별 주문이 트레이딩 시스템 내에서 추적 가능하도록 보장합니다.  

또한 `nextValidOrderId`는 주문의 무결성을 유지하고, 동시에 또는 빠른 연속으로 제출된 여러 주문 간의 중복을 방지하는 역할을 합니다. 트레이딩 앱이 IB API에 연결되면 서버에서 `nextValidOrderId`라는 정수 변수를 받아오며, 이는 **각 클라이언트의 TWS 연결에서 유일한 값**입니다. 첫 번째 주문을 제출할 때 반드시 이 ID를 사용해야 하며, 이후에는 각 새로운 주문에 대해 사용자가 이 식별자를 증가시키는 역할을 담당해야 합니다.  

---

### **준비하기**  

우리는 이미 `client.py` 및 `wrapper.py` 파일을 **trading-app** 디렉터리에 생성했다고 가정합니다. 만약 생성하지 않았다면 지금 생성하세요.  

---

### **어떻게 하는지...**  

먼저, `wrapper.py` 파일에서 정수형 `nextValidOrderId` 를 처리하는 코드를 추가하고, 주문 실행의 세부 정보를 출력하겠습니다.  

1. `IBWrapper` 클래스의 `__init__` 메서드에 `nextValidOrderId` 인스턴스 변수를 추가합니다:  

   ```python
   self.nextValidOrderId = None
   ```

2. `IBWrapper` 클래스에서 상속받은 `EWrapper` 클래스의 메서드를 재정의하여 `nextValidId` 메서드를 구현합니다:  

   ```python
   def nextValidId(self, order_id):
       super().nextValidId(order_id)
       self.nextValidOrderId = order_id
   ```

3. `EWrapper` 클래스에서 상속받은 `orderStatus` 메서드를 재정의하여 구현합니다:  

   ```python
   def orderStatus(
       self,
       order_id,
       status,
       filled,
       remaining,
       avg_fill_price,
       perm_id,
       parent_id,
       last_fill_price,
       client_id,
       why_held,
       mkt_cap_price,
   ):
       print(
           "orderStatus - orderid:", order_id,
           "status:", status,
           "filled", filled,
           "remaining", remaining,
           "lastFillPrice", last_fill_price,
       )
   ```

4. `EWrapper` 클래스에서 상속받은 `openOrder` 메서드를 재정의하여 구현합니다:  

   ```python
   def openOrder(self, order_id, contract, order, order_state):
       print(
           "openOrder id:", order_id,
           contract.symbol,
           contract.secType,
           "@",
           contract.exchange,
           ":",
           order.action,
           order.orderType,
           order.totalQuantity,
           order_state.status,
       )
   ```

5. `execDetails` 메서드 추가
상속받은 `EWrapper` 클래스에서 `execDetails` 메서드를 재정의하여 구현합니다:

    ```python
    def execDetails(self, request_id, contract, execution):
        print(
            "Order Executed: ",
            request_id,
            contract.symbol,
            contract.secType,
            contract.currency,
            execution.execId,
            execution.orderId,
            execution.shares,
            execution.lastLiquidity,
        )
    ```

6. `client.py` 파일 수정
이제 `client.py` 파일을 열어 `__init__` 메서드 아래에 `send_order` 라는 사용자 정의 메서드를 추가합니다.  
이 메서드는 `contract` 객체와 `order` 객체를 받아 `nextValidOrderId` 변수를 증가시키고, 거래소에 주문을 보냅니다.

    ```python
    def send_order(self, contract, order):
        order_id = self.wrapper.nextValidOrderId
        self.placeOrder(orderId=order_id, contract=contract, order=order)
        self.reqIds(-1)
        return order_id
    ```

변경된 `client.py` 코드 예시, 변경된 내용은 다음과 같습니다:

```python
import time
import pandas as pd
from utils import Tick, TRADE_BAR_PROPERTIES
from ibapi.client import EClient

class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

    def send_order(self, contract, order):
        order_id = self.wrapper.nextValidOrderId
        self.placeOrder(orderId=order_id, contract=contract, order=order)
        self.reqIds(-1)
        return order_id
```

---

### **작동 방식**

먼저, `IBWrapper` 클래스에서 재정의된 메서드들을 다룹니다.  

주문이 `IBClient` 클래스의 `placeOrder` 메서드를 통해 제출되면, `IBWrapper` 클래스의 `orderStatus`, `openOrder`, `execDetails` 메서드가 주문의 수명 주기에 따라 호출됩니다.  

이 메서드들은 다음과 같은 역할을 수행합니다:  

- `orderStatus`: 제출된 주문의 상태 업데이트를 수신합니다.  
- `openOrder`: 제출되었으나 아직 완전히 실행되지 않은 주문 정보를 제공합니다.  
- `execDetails`: 주문 실행에 대한 세부 정보를 제공합니다.  

각 콜백 메서드는 IB API에서 전달된 매개변수를 받아 처리합니다. 이 예제에서는 정보를 단순히 출력하지만, 보다 정교한 애플리케이션에서는 이 이벤트들을 활용하여 **위험 분석, 포트폴리오 업데이트, 알림 시스템 등을 구현할 수 있습니다.**  

`send_order` 메서드가 호출되면, `contract` 객체와 `order` 객체를 받습니다.  
- `contract` 객체는 주문할 금융 상품을 나타냅니다.  
- `order` 객체는 주문의 세부 정보를 포함합니다.  
- `order_id = self.wrapper.nextValidOrderId` 는 `IBWrapper` 객체에서 **다음 유효한 주문 ID** 를 검색합니다.  
- `self.placeOrder(orderId=order_id, contract=contract, order=order)` 는 IB API의 `placeOrder` 메서드를 호출하여 주문을 실행합니다.  
- `self.reqIds(-1)` 은 다음 유효한 주문 ID를 위한 내부 카운터를 서버에서 증가시키도록 요청합니다.  

---

### **더 알아보기...**  

이제 Python을 사용하여 IB API를 통해 주문을 전송할 준비가 모두 완료되었습니다.  
주문을 실행하려면, `app.py` 파일의 상단에서 `order` 유형과 `BUY` 상수를 가져옵니다:

```python
from order import limit, BUY
```

그런 다음, 트레이딩 앱을 정의하는 줄 이후에 다음 코드를 추가합니다:

```python
limit_order = limit(BUY, 100, 190.00)
app.send_order(aapl, limit_order)
```

이 코드는 **IB API를 사용하여 계약 객체(Contract Object)** 를 생성하는 **"Creating a Contract object with the IB API"** 단계에서 설정한 `contract` 객체와,  
**IB API를 사용하여 주문 객체(Order Object)** 를 생성하는 **"Creating an Order object with the IB API"** 단계에서 설정한 `order` 객체를 활용합니다.  
(`Chapter 9` 참고)

변경된 내용은 다음과 같습니다:

```python
import threading
import time
import sqlite3
from wrapper import IBWrapper
from client import IBClient
from contract import stock, future, option
from order import limit, BUY

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.create_table()
        self.connect(ip, port, client_id)
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        time.sleep(2)

<snip>

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=11)

    aapl = stock("AAPL", "SMART", "USD")
    gbl = future("GBL", "EUREX", "202403")
    pltr = option("PLTR", "BOX", "20240315", 20, "C")

    limit_order = limit(BUY, 100, 190.00)

    app.send_order(aapl, limit_order)
    time.sleep(30)

    app.disconnect()
```

이 코드를 실행하면, 터미널에서 일련의 메시지를 확인할 수 있습니다.  
이 메시지들은 `placeOrder` 메서드가 호출된 후 **`orderStatus`, `openOrder`, `execDetails`** 콜백이 실행되면서 출력됩니다.

<img src="./images/fig_11_01.png" width=800>

그림 11.1: 지정가 주문이 체결되었음을 나타내는 메시지

>**중요한 참고 사항 (IMPORTANT NOTE)**  
>
>이 예제에서는 현재 매도 호가(ask)보다 높은 가격으로 **매수 지정가 주문(buy limit order)** 을 입력합니다. 작성 당시의 매도 호가는 **$188.66** 이었습니다. 따라서, 비록 지정가 주문(limit order)이라고 하더라도 즉시 체결(executed)됩니다. 이 작업은 **데모 및 테스트 목적**으로만 수행됩니다. 일반적인 시나리오에서는 **최적 매수 호가(best bid) 아래에 지정가 주문을 입력하여 원하는 가격을 기다리는 방식**을 사용합니다.

---

### **추가 참고 사항**  

더 많은 정보를 확인하려면, 유명한 **`nextValidOrderId`** 및 주문 실행(order execution)에 대한 세부 정보를 아래 문서에서 확인하세요:  

🔗 [Interactive Brokers API 문서 – 주문 제출 (order submission)](https://interactivebrokers.github.io/tws-api/order_submission.html)  

이 URL에서는 **`openOrder`** 및 **`orderStatus`** 콜백에 대한 자세한 내용을 설명합니다.

---

## **주문이 실행된 후 관리하기 (Managing orders once they’re placed)**  

효과적인 주문 관리는 중요하며, 일반적으로 기존 주문을 **취소**하거나 **업데이트**하는 작업을 포함합니다.  

주문을 취소하는 것은 간단합니다. 예를 들어, 더 이상 필요하지 않은 **지정가 주문(limit order) 또는 손절매 주문(stop loss order)** 을 입력했을 경우, 시장 상황이 변하거나 전략이 다른 진입 또는 청산 포지션을 지시할 때, **IB API를 사용하여 완전히 주문을 취소할 수 있습니다.**  

반면에, 주문을 주문 장부(order book)에 남겨두되 **속성을 변경**하고 싶을 수도 있습니다. 트레이더들은 **거래 수량을 변경**하여 시장 분석 또는 위험 관리 요구 사항에 따라 포지션을 조정하기 위해 주문을 자주 업데이트합니다. 또한, **지정가(limit price)를 조정하는 것은 매우 일반적인 업데이트 방법**으로, 시장 상황에 따라 **최대 매수 가격 또는 최소 매도 가격을 새롭게 설정**할 수 있습니다. 마찬가지로, **손절매 가격(stop price)을 변경하는 것**은 잠재적인 손실을 관리하거나 수익을 확보하는 **전략적 움직임**이며, 특히 변동성이 큰 시장에서 유용합니다.  

IB API를 사용하여 **기존 주문을 업데이트하려면**, 기존 주문과 동일한 필드 값을 사용하되 변경할 **매개변수(parameter)** 를 지정하여 **`placeOrder`** 메서드를 호출하면 됩니다. 이때, 주문의 **ID(order ID)** 는 반드시 기존의 오픈 주문과 **일치해야 합니다.** IB(Interactive Brokers)는 **주문 가격(price), 크기(size), 유효 시간(time in force)만 변경하는 것을 권장**합니다.  

그러나 **주문 세부 정보를 추적하는 것은 어렵기 때문에**, **변경하려는 주문을 취소한 후, 새롭게 업데이트된 매개변수로 다시 입력하는 방법이 더 간단합니다.** 이 레시피에서는 **이러한 방식**을 사용할 것입니다.

---

### **준비하기**  

우리는 **`client.py`** 및 **`app.py`** 파일이 **`trading-app`** 디렉터리에 이미 생성되어 있다고 가정합니다.  
만약 아직 생성하지 않았다면, 지금 생성하세요.  

---

### **어떻게 하는지...**  

`client.py` 파일에 **주문을 관리하기 위한 3가지 새로운 메서드**를 추가할 것입니다:

1. `IBClient` 클래스의 `__init__` 메서드 바로 아래에 `cancel_all_orders` 메서드를 추가합니다:

   ```python
   def cancel_all_orders(self):
       self.reqGlobalCancel()
   ```

2. 다음으로, `cancel_order_by_id` 메서드를 추가합니다:

   ```python
   def cancel_order_by_id(self, order_id):
       self.cancelOrder(orderId=order_id, manualCancelOrderTime="")
   ```

3. 마지막으로, `update_order` 메서드를 추가합니다:

   ```python
   def update_order(self, contract, order, order_id):
       self.cancel_order_by_id(order_id)
       return self.send_order(contract, order)
   ```

4. 변경된 `client.py` 파일의 결과는 다음과 같습니다:

   ```python
   
   class IBClient(EClient):
       def __init__(self, wrapper):
           EClient.__init__(self, wrapper)

       def cancel_all_orders(self):
           self.reqGlobalCancel()

       def cancel_order_by_id(self, order_id):
           self.cancelOrder(orderId=order_id, manualCancelOrderTime="")

       def update_order(self, contract, order, order_id):
           self.cancel_order_by_id(order_id)
           return self.send_order(contract, order)
   
   ```

---

### **작동 방식**

우리는 먼저 **모든 열린 주문을 취소하는 함수**를 생성합니다.  
**`cancel_all_orders`** 메서드는 **`reqGlobalCancel`** 메서드를 실행하는데,  
이 명령은 현재 세션에서 **모든 열린 주문을 취소**하여  
트레이딩 시스템에 **미결 주문(pending orders)이 남아있지 않도록 보장**합니다.  

>**중요한 참고 사항 (IMPORTANT NOTE)**  
>
>`cancel_all_orders` 를 호출하면 **모든 열린 주문이 취소됩니다.** 이것은 **TWS(Trader Workstation)에서 수동으로 입력된 주문뿐만 아니라, IB API를 통해 입력된 주문도 포함하여 취소됨을 의미합니다.**  

특정 **단일 주문(single order)을 취소하려면** `cancel_order_by_id` 메서드를 사용합니다. 이 메서드는 **정수형 `order_id` 를 통해 특정 주문을 취소**합니다.  

해당 메서드가 실행되면, `cancelOrder` 메서드를 호출하며 매개변수로 **정수형 `order_id`** 와 **빈 문자열(`manualCancelOrderTime=""`)** 을 전달합니다.  

또한, `send_order` 메서드는 **취소된 주문의 `order_id` 를 반환**합니다.  

주문을 업데이트하려면 **두 가지 메서드를 결합**합니다.  
- **`update_order`** 메서드는 먼저 **`cancel_order_by_id`** 메서드를 사용하여 **기존 주문(`order_id`)을 취소**합니다.  
- 그런 다음, **`send_order`** 메서드를 사용하여 **새로운 주문을 생성 및 전송**합니다.  
- 이 과정에서 **기존 주문을 새로운 주문으로 대체(replacing the original order)** 하게 됩니다.  

**이것이 IB API를 사용하여 주문을 업데이트하는 가장 권장되는 방법입니다.**  

---

### **더 알아보기...**  

이제 우리가 만든 새로운 메서드를 테스트해 보겠습니다.  
 `app.py` 파일에서 **AAPL 주문을 정의하는 줄 이후**에 다음 코드를 추가하세요:

```python
order_1 = limit(BUY, 10, 185.0)
order_1_id = app.send_order(aapl, order_1)
```

✅ **이 코드를 실행하면**, TWS(Trader Workstation)의 **Orders 섹션에서 주문을 확인할 수 있습니다.**  

<img src="./images/fig_11_02.png" width=800>
 
그림 11.2: 시장에서 안전하게 대기 중인 AAPL 지정가 주문
 
주문을 취소하려면 다음 코드를 실행하세요:
 
```python
app.cancel_order_by_id(order_1_id)
```
 
이제 주문이 취소된 것을 확인할 수 있습니다:

<img src="./images/fig_11_03.png" width=800>
 
그림 11.3: AAPL 지정가 주문이 취소됨
 
주문을 다시 입력하고, 다른 지정가로 두 번째 주문을 생성한 뒤 업데이트해 보겠습니다:
 
```python
order_2 = limit(BUY, 10, 187.50)
app.update_order(aapl, order_2, order_1_id) 
```
 
원래 주문은 취소되고 새로운 주문이 대기 중입니다:
 
<img src="./images/fig_11_04.png" width=800>
 
그림 11.4: 원래 AAPL 주문이 취소되고 새로운 AAPL 주문이 입력됨
 
마지막으로 모든 미체결 주문을 취소합니다:
 
```python
app.cancel_all_orders() 
```
 
결과적으로 모든 미체결 주문이 취소됩니다:
 
<img src="./images/fig_11_05.png" width=800>

그림 11.5: 모든 미체결 주문이 취소됨


---

### **추가 참고 사항**  

주문 수정에 대한 자세한 내용은 다음 링크를 참조하세요: https://interactivebrokers.github.io/tws-api/modifying_orders.html.

---

## **포트폴리오 세부 정보 가져오기 (Getting details about your portfolio)**  

IB API는 포트폴리오 데이터를 **한 번의 API 호출**을 통해 **총 157개의 서로 다른 포트폴리오 값**을 반환하는 **종합적인 스냅샷(snapshot)** 을 제공합니다.  
이 데이터는 **다양한 지표(metrics)와 데이터 포인트(data points)** 를 포함하여 포트폴리오에 대한 **세부적인 뷰(view)** 를 제공합니다.  

**`updateAccountValue`** 메서드를 통해 제공되는 **계좌 값(account values)** 은 다음과 같이 분류될 수 있습니다:  

- **원자재(Commodities):** `-c` 접미사가 붙음  
- **유가 증권(Securities):** `-s` 접미사가 붙음  
- **총계(Totals):** 접미사 없음  

**이 레시피에서는 위의 데이터 포인트를 가져오는 코드를 작성할 것입니다.**  

---

### **준비하기**  

우리는 **`client.py`, `wrapper.py`, `app.py`** 파일이 **`trading-app`** 디렉터리에 이미 생성되어 있다고 가정합니다.  
만약 아직 생성하지 않았다면, 지금 생성하세요.  

---

### **어떻게 하는지...**  

이 작업의 첫 번째 단계는 **계좌 번호(account number)** 를 `IBApp` 클래스에 포함시키는 것입니다.  
- 단일 계좌 구조에서는 계좌 번호가 **필수가 아니지만**,  
  **여러 개의 계좌를 사용하는 경우에는 계좌 번호를 지정하는 것이 가장 좋은 방법(best practice)** 입니다.  

그다음,  
1. **콜백(callback)** 을 `IBWrapper` 클래스에 추가합니다.  
2. **요청 메서드(request method)** 를 `IBClient` 클래스에 추가합니다.  

1. `IBApp` 클래스의 `__init__` 메서드 수정
계좌 번호(`account`)를 매개변수로 추가합니다:

```python
def __init__(self, ip, port, client_id, account):
```
2. `IBWrapper` 클래스의 `__init__` 메서드 수정
다음 **딕셔너리(dictionary)** 를 추가하여 계좌 정보를 저장합니다:

```python
self.account_values = {}
```
3. `TWS` 메시지에 응답하는 콜백(callback) 함수 추가
📌 **`updateAccountValue` 메서드** 를 `IBWrapper` 클래스에 추가합니다:

```python
def updateAccountValue(self, key, val, currency, account):
    try:
        val_ = float(val)
    except:
        val_ = val
    self.account_values[key] = (val_, currency)
```

변경된 `wrapper.py` 코드 예시

```python
import threading
from ibapi.wrapper import EWrapper

class IBWrapper(EWrapper):
    def __init__(self):
        EWrapper.__init__(self)
        self.nextValidOrderId = None
        self.historical_data = {}
        self.market_data = {}
        self.streaming_data = {}
        self.stream_event = threading.Event()
        self.account_values = {}

    <snip>

    def updateAccountValue(self, key, val, currency, account):
        try:
            val_ = float(val)
        except:
            val_ = val
        self.account_values[key] = (val_, currency)
```

4. `IBClient` 클래스 끝에 `get_account_values` 메서드 추가

```python
def get_account_values(self, key=None):
    self.reqAccountUpdates(True, self.account)
    time.sleep(2)
    if key:
        return self.account_values[key]
    return self.account_values
```

---


### **작동 방식**  

이제 우리는 **일반적인 패턴**을 따르고 있습니다.  
즉, **`IBClient` 클래스에서 요청(request)을 실행하고**, 그 결과를 **`IBWrapper` 클래스의 딕셔너리(dictionary)에 저장하는 방식**입니다.  

- `updateAccountValue` 메서드는 **콜백(callback) 함수**로서,  
  특정 계좌 속성을 나타내는 **키(key), 값(value), 통화(currency), 계좌 식별자(account identifier)** 를 받아  
  이를 **딕셔너리(account_values)** 에 `(값, 통화)` 형태의 **튜플(tuple)** 로 저장합니다.  

**`get_account_values` 메서드의 동작 방식**  

1. **계좌 업데이트 요청(reqAccountUpdates) 실행**  
2. **계좌 데이터(account data) 업데이트를 기다리기 위해 2초간 실행을 일시 정지**  
3. **특정 키(key)가 제공되었을 경우**, 해당 **키에 연결된 값**을 `account_values` 딕셔너리에서 반환  
4. **키가 제공되지 않은 경우**, 전체 **`account_values` 딕셔너리를 반환**  
   - 이는 **모든 계좌 관련 값(account-related values)의 스냅샷(snapshot)** 을 제공합니다.  

**IB API는 총 157개의 서로 다른 계좌 값을 반환합니다.**  
- 아래는 **가장 일반적인 속성 값**입니다:

  - **AvailableFunds**  
  - **BuyingPower**  
  - **CashBalance**  
  - **Currency**  
  - **EquityWithLoanValue**  
  - **FullAvailableFunds**  
  - **NetLiquidation**  
  - **RealizedPnL**  
  - **TotalCashBalance**  
  - **UnrealizedPnL**  

---

### **더 알아보기...**  

**계좌의 "순 청산 가치(Net Liquidation Value)" 를 가져오는 방법**  
`app.py` 파일에서 **`IBApp` 클래스 이후의 코드를 아래 코드로 교체하세요**:

```python
if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10, account="DU7129120")
    
    account_values = app.get_account_values()
    net_liquidation = app.get_account_values("NetLiquidation")
    
    app.disconnect()
```

**위 코드를 실행하면**, **총 157개의 계좌 값**을 포함하는 **`account_values` 딕셔너리(dictionary)** 를 얻을 수 있습니다.  
아래는 그 예시입니다:  

<img src="./images/fig_11_06.png" width=500>

그림 11.6: get_account_values가 제공하는 데이터의 예시

결과는 계좌의 순 청산 가치와 해당 가치가 표시되는 통화를 포함하는 net_liquidation 튜플입니다. 다음과 같이 표시됩니다:

<img src="./images/fig_11_07.png" width=200>

그림 11.7: 계좌의 순 청산 가치

---

### **추가 참고 사항**  

계좌 값과 설명의 전체 목록은 다음 URL을 참조하세요: https://interactivebrokers.github.io/tws-api/interfaceIBApi_1_1EWrapper.html#ae15a34084d9f26f279abd0bdeab1b9b5.

---

## **포지션 및 포지션 세부 정보 조사**

IB API에서 포지션 수준의 세부 정보를 가져오기 위해 포지션 크기, 시장 가격, 가치, 평균 비용 및 PnL과 같은 데이터를 포함하여 특정 API 호출을 활용할 수 있습니다. 이러한 호출은 계정에 보유된 각 포지션에 대한 세부 정보를 요청하며, API는 요청된 데이터를 각 포지션별로 응답합니다. 이를 통해 우리의 보유 자산을 포괄적으로 파악할 수 있습니다. 이 레시피에서는 포지션 데이터를 가져오는 방법을 소개합니다. 다음 장에서는 이를 활용하여 트레이딩 전략을 구축할 것입니다.

---

### **준비하기**

우리는 사용자가 `client.py`, `wrapper.py`, `app.py` 파일을 `trading-app` 디렉터리에 생성했다고 가정합니다. 만약 아직 생성하지 않았다면, 지금 생성하세요.

---

### **어떻게 하는지...**  

이전 레시피인 **포트폴리오에 대한 세부 정보 가져오기**에서, 우리는 `IBClient` 클래스의 `reqAccountUpdates` 요청 메서드를 사용하여 계정 세부 정보를 요청했습니다. `reqAccountUpdates`를 호출하면 두 개의 콜백이 트리거됩니다. 첫 번째는 `updateAccountValue`이며, 우리는 이를 `IBWrapper` 메서드에서 오버라이드했습니다. 이 메서드는 계정에 대한 세부 정보를 반환합니다. `reqAccountUpdates`는 또한 계정 내 포지션에 대한 세부 정보를 반환하는 `updatePortfolio` 콜백을 트리거합니다. 우리는 같은 `reqAccountUpdates` 메서드를 사용하여 `updatePortfolio` 메서드를 활용해 계정 세부 정보를 가져올 것입니다.

1. `IBWrapper` 클래스의 `__init__` 메서드에 포지션 세부 정보를 저장할 딕셔너리를 추가합니다.

   ```python
   self.positions = {}
   ```

2. `IBWrapper` 클래스의 끝부분에 다음 메서드를 추가합니다.

   ```python
   def updatePortfolio(
       self,
       contract,
       position,
       market_price,
       market_value,
       average_cost,
       unrealized_pnl,
       realized_pnl,
       account_name
   ):
       portfolio_data = {
           "contract": contract,
           "symbol": contract.symbol,
           "position": position,
           "market_price": market_price,
           "market_value": market_value,
           "average_cost": average_cost,
           "unrealized_pnl": unrealized_pnl,
           "realized_pnl": realized_pnl,
       }

       self.positions[contract.symbol] = portfolio_data
   ```

3. 변경된 결과는 `wrapper.py` 파일의 다음 코드와 같습니다.

   ```python
   class IBWrapper(EWrapper):
       def __init__(self):
           Wrapper.__init__(self)
           self.nextValidOrderId = None
           self.historical_data = {}
           self.streaming_data = {}
           self.stream_event = threading.Event()
           self.account_values = {}
           self.positions = {}

       def updatePortfolio(
           self,
           contract,
           position,
           market_price,
           market_value,
           average_cost,
           unrealized_pnl,
           realized_pnl,
           account_name
       ):
           portfolio_data = {
               "contract": contract,
               "symbol": contract.symbol,
               "position": position,
               "market_price": market_price,
               "market_value": market_value,
               "average_cost": average_cost,
               "unrealized_pnl": unrealized_pnl,
               "realized_pnl": realized_pnl,
           }

           self.positions[contract.symbol] = portfolio_data
   ```

4. 이제 `IBClient` 클래스의 끝부분에 다음 메서드를 추가합니다.

   ```python
   def get_positions(self):
       self.reqAccountUpdates(True, self.account)
       time.sleep(1)
       return self.positions
   ```

---   

### **작동 방식**  

`reqAccountUpdates`가 호출되면, `updatePortfolio` 콜백이 계정 내 모든 포지션에 대해 트리거됩니다. TWS는 포지션에 대한 세부 정보를 메서드로 전달하며, 이 정보는 `positions` 딕셔너리에 저장되고, 계약(symbol)별로 키가 지정됩니다.

`get_positions` 메서드는 `reqAccountUpdates` 메서드를 통해 콜백을 트리거하고, 2초 동안 대기한 후 포지션을 포함하는 딕셔너리를 반환합니다.

---

### **더 알아보기...**

계정의 현재 포지션을 가져오려면, `app.py` 파일의 끝에 다음 코드를 추가하십시오.

```python
positions = app.get_positions()
```

결과는 포지션 세부 정보를 포함하는 딕셔너리입니다.

<img src="./images/fig_11_08.png" width=800>

그림 11.8: 계정의 포지션 세부 정보

---

### **추가 참고 사항**  

reqAccountsUpdates 메서드를 사용한 포지션 데이터 가져오기에 대해 다음 URL에서 자세히 알아보세요: https://interactivebrokers.github.io/tws-api/account_updates.html.

---


## **포트폴리오 손익 계산**

IB API에서 포트폴리오 PnL(손익) 정보를 얻기 위해 특정 API 호출을 활용할 수 있습니다. 이러한 호출은 총 일일 손익, 총 미실현 손익, 총 실현 손익을 요청하며, API는 요청된 데이터와 함께 응답합니다. PnL을 가져오면 주기적인 포트폴리오 수익률을 계산할 수 있으며, 이는 다시 포트폴리오 수익률 기반의 다양한 리스크 메트릭을 활용하는 데 도움이 됩니다. 다음 장에서는 달러 기준 손익을 활용하여 주기적인 포트폴리오 수익률을 계산하는 방법을 다룰 것입니다. 이 레시피에서는 포트폴리오 손익을 요청하고 수신하는 데 집중합니다.

---

### **준비하기**

우리는 사용자가 `client.py`, `wrapper.py`, `app.py` 파일을 `trading-app` 디렉터리에 생성했다고 가정합니다. 만약 아직 생성하지 않았다면, 지금 생성하세요.

---

### **어떻게 하는지...**  

이전과 동일한 패턴을 따릅니다.

1. `IBWrapper` 클래스의 `__init__` 메서드 끝에 포지션 세부 정보를 저장할 딕셔너리를 추가합니다.

   ```python
   self.account_pnl = {}
   ```

2. `IBWrapper` 클래스의 끝부분에 다음 메서드를 추가합니다.

   ```python
   def pnl(self, request_id, daily_pnl, unrealized_pnl, realized_pnl):
       pnl_data = {
           "daily_pnl": daily_pnl,
           "unrealized_pnl": unrealized_pnl,
           "realized_pnl": realized_pnl
       }

       self.account_pnl[request_id] = pnl_data
   ```

3. 변경된 결과는 `wrapper.py` 파일의 다음 코드와 같습니다.

   ```python
   <snip>
   class IBWrapper(EWrapper):
       def __init__(self):
           EWrapper.__init__(self)
           self.nextValidOrderId = None
           self.historical_data = {}
           self.streaming_data = {}
           self.stream_event = threading.Event()
           self.account_values = {}
           self.positions = {}
           self.account_pnl = {}

       def pnl(self, request_id, daily_pnl, unrealized_pnl, realized_pnl):
           pnl_data = {
               "daily_pnl": daily_pnl,
               "unrealized_pnl": unrealized_pnl,
               "realized_pnl": realized_pnl
           }

           self.account_pnl[request_id] = pnl_data
   <snip>
   ```

4. 이제 `IBClient` 클래스의 끝부분에 다음 메서드를 추가합니다.

   ```python
   def get_pnl(self, request_id):
       self.reqPnL(request_id, self.account, "")
       time.sleep(2)
       return self.account_pnl
   ```

---   

### **작동 방식**

`pnl` 메서드는 PnL 데이터를 처리하는 콜백 함수입니다. 이 함수가 호출되면, 정수 `request_id`와 함께 세 가지 유형의 PnL 데이터(`daily_pnl`, `unrealized_pnl`, `realized_pnl`)를 받습니다. 그런 다음, 이 값들로 `pnl_data` 딕셔너리를 생성하고, 이를 `account_pnl` 딕셔너리에 `request_id`를 키로 저장하여 해당 요청과 연결된 계정의 PnL 정보를 업데이트합니다.

`get_pnl` 메서드는 특정 계정의 PnL 데이터를 요청합니다. 이 메서드는 `reqPnL`을 호출하여 주어진 `request_id` 및 계정과 연결된 PnL 정보를 요청한 후, 데이터가 수신 및 처리될 시간을 확보하기 위해 2초 동안 대기합니다. 그런 다음, `account_pnl`에 저장된 업데이트된 PnL 데이터를 반환합니다.  

미실현 손익은 아직 현금화되지 않은 투자에 대한 잠재적 금융 이익 또는 손실을 의미합니다. 반면, 실현된 손익은 자산이 매도된 후 발생한 실제 이익 또는 손실입니다.

---

### **더 알아보기...**  

`app.py` 파일의 끝에 다음 코드를 추가하여 계정의 PnL을 가져옵니다.

```python
pnl = app.get_pnl(request_id=99)
```

결과는 일일 PnL, 미실현 PnL 및 실현 PnL을 포함하는 딕셔너리입니다.

<img src="./images/fig_11_09.png" width=800>

그림 11.9: get_pnl을 호출한 결과는 계정 PnL 정보가 포함된 딕셔너리입니다

---

### **추가 참고 사항**  

IB API는 두 가지 다른 소스에서 PnL 데이터를 가져올 수 있습니다: TWS 계정 창과 TWS 포트폴리오 창입니다. 이러한 소스들은 서로 다른 업데이트 시간을 가지고 있어 포트폴리오 PnL 값이 다를 수 있습니다. 이 레시피에서 설명한 방법은 전역 구성에 지정된 리셋 일정에 따라 TWS 포트폴리오 창에서 PnL 데이터를 요청합니다.

포트폴리오 PnL이 어떻게 계산되는지에 대한 자세한 내용은 다음 URL을 참조하세요: https://interactivebrokers.github.io/tws-api/pnl.html.